In [1]:
# Importing dependencies
import pandas as pd
import numpy as np
import os
import pickle
import random

In [4]:
# Define the path to the CSV file
csv_file = 'hobby_dataset_original.csv'
rated_file = 'hobby_dataset_rated.csv'

# Run if rated file does not exist already
if (not rated_file):
    # Check if original hobby dataset is available
    if os.path.isfile(csv_file):
        # Load the dataset into a pandas dataframe
        df = pd.read_csv(csv_file)

        # Check if the base_rating column already exists in the dataframe
        if 'base_rating' not in df.columns:
            # Assign a base rating of 15 to each hobby in the dataframe
            df['base_rating'] = 15

            # Save the rated dataset to a CSV file
            df.to_csv('hobby_dataset_rated.csv', index=False)